In [1]:
import time
from database.elasticsearch import es
from tags.search import tags_from_index, aggregate_tags, sort_by_statistics

{'_scroll_id': 'DXF1ZXJ5QW5kRmV0Y2gBAAAAAAAAAG4WOVpaVk9TX25RQmVhNU5SRTV5Z05yQQ==',
 'took': 19,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 530, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'tasks',
    '_type': '_doc',
    '_id': 'l5CC8m8BTANBIwoUsx-k',
    '_score': 1.0,
    '_source': {}},
   {'_index': 'tasks',
    '_type': '_doc',
    '_id': 'mJCC8m8BTANBIwoUsx-k',
    '_score': 1.0,
    '_source': {}},
   {'_index': 'tasks',
    '_type': '_doc',
    '_id': 'mZCC8m8BTANBIwoUsx-l',
    '_score': 1.0,
    '_source': {}},
   {'_index': 'tasks',
    '_type': '_doc',
    '_id': 'mpCC8m8BTANBIwoUsx-l',
    '_score': 1.0,
    '_source': {}},
   {'_index': 'tasks',
    '_type': '_doc',
    '_id': 'm5CC8m8BTANBIwoUsx-l',
    '_score': 1.0,
    '_source': {}},
   {'_index': 'tasks',
    '_type': '_doc',
    '_id': 'nJCC8m8BTANBIwoUsx-l',
    '_score': 1.0,
    '_source': {}},
   {'_index': 'tasks',
 

In [40]:
es.search(index='tasks',_source_includes=['_id'],size=1000,body={})

{'took': 34,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 530, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'tasks',
    '_type': '_doc',
    '_id': 'l5CC8m8BTANBIwoUsx-k',
    '_score': 1.0,
    '_source': {}},
   {'_index': 'tasks',
    '_type': '_doc',
    '_id': 'mJCC8m8BTANBIwoUsx-k',
    '_score': 1.0,
    '_source': {}},
   {'_index': 'tasks',
    '_type': '_doc',
    '_id': 'mZCC8m8BTANBIwoUsx-l',
    '_score': 1.0,
    '_source': {}},
   {'_index': 'tasks',
    '_type': '_doc',
    '_id': 'mpCC8m8BTANBIwoUsx-l',
    '_score': 1.0,
    '_source': {}},
   {'_index': 'tasks',
    '_type': '_doc',
    '_id': 'm5CC8m8BTANBIwoUsx-l',
    '_score': 1.0,
    '_source': {}},
   {'_index': 'tasks',
    '_type': '_doc',
    '_id': 'nJCC8m8BTANBIwoUsx-l',
    '_score': 1.0,
    '_source': {}},
   {'_index': 'tasks',
    '_type': '_doc',
    '_id': 'nZCC8m8BTANBIwoUsx-l',
    '_score': 1.0,
    '_sou

In [2]:
task_fields=['task','task_description','project','project_description']
now = time.time()
task_tags = tags_from_index(es,'tasks',task_fields)
aggregated_task_tags = aggregate_tags(task_tags)
sorted_task_tags = sort_by_statistics(aggregated_task_tags)

print(time.time()-now)

0.344301700592041


In [14]:
sorted_task_tags

[(361, 3.7516778523490126, 149, 261, 'community.nxp.com'),
 (260, 1.7567567567567601, 148, 260, 'thread'),
 (100, 2.000000000000001, 50, 100, 'l2sup'),
 (100, 2.000000000000001, 50, 100, 'groups'),
 (90, 2.000000000000001, 45, 90, 'hidglobal'),
 (86, 2.9999999999999964, 16, 16, 'support'),
 (53, 3.0000000000000044, 5, 5, 'boards'),
 (47, 1.9999999999999964, 2, 2, 'bring'),
 (46, 3.0000000000000013, 28, 28, 'driver'),
 (45, 2.999999999999999, 22, 22, 'schematic'),
 (40, 3.1666666666666647, 6, 7, 'bsp'),
 (36, 3.0000000000000004, 16, 16, 'pcb'),
 (34, 2.999999999999999, 16, 16, 'layout'),
 (31, 3.2000000000000006, 10, 12, 'nxp'),
 (30, 2.9999999999999996, 9, 9, 'linux'),
 (30, 2.166666666666667, 23, 23, 'test'),
 (24, 3.0, 12, 12, 'mini'),
 (23, 2.125, 8, 9, 'custom'),
 (22, 2.555555555555554, 18, 19, 'boot'),
 (21, 2.0, 15, 15, 'lpc5500'),
 (21, 1.9999999999999993, 11, 11, 'integration'),
 (20, 2.4999999999999982, 17, 17, 'camera'),
 (19, 0.9999999999999996, 19, 19, 'usb'),
 (18, 4.0, 6

In [77]:
thread_fields=['text']

now=time.time()
thread_tags = (tag for thread_ids in scroll_all_ids(es,'threads') 
             for tag in generate_combined_tags(index='threads', 
                               ids=thread_ids, fields=thread_fields))
print(time.time() - now)
aggregated_thread_tags = aggregate_tags(thread_tags)
print(time.time() - now)
sorted_thread_tags =  sort_by_statistics(aggregated_thread_tags)
print(time.time() - now)

0.00024008750915527344
0.5367350578308105
0.5594425201416016


In [78]:
sorted_thread_tags

[(636, 22.71428571428571, 28, 636, 'usb'),
 (523, 12.756097560975602, 41, 523, 'boot'),
 (463, 9.259999999999996, 50, 463, 'device'),
 (450, 7.758620689655174, 58, 450, 'board'),
 (244, 5.545454545454545, 44, 244, 'linux'),
 (244, 4.518518518518517, 54, 244, 'support'),
 (234, 13.764705882352938, 17, 234, 'm4'),
 (227, 28.375, 8, 227, 'tee'),
 (221, 15.785714285714283, 14, 221, 'gpio'),
 (215, 6.515151515151514, 33, 215, 'kernel'),
 (207, 6.272727272727271, 33, 207, 'image'),
 (206, 8.583333333333336, 24, 206, 'schematic'),
 (205, 8.541666666666668, 24, 205, 'port'),
 (203, 5.970588235294117, 34, 203, 'design'),
 (201, 5.432432432432434, 37, 201, 'test'),
 (201, 4.56818181818182, 44, 201, 'code'),
 (198, 3.8076923076923066, 52, 198, 'nxp'),
 (189, 5.399999999999998, 35, 189, 'c'),
 (163, 32.6, 5, 163, 'pcie'),
 (163, 32.6, 5, 163, 'caam'),
 (163, 13.583333333333336, 12, 163, 'storage'),
 (162, 5.586206896551722, 29, 162, 'build'),
 (158, 11.285714285714286, 14, 158, 'host'),
 (151, 7.9

In [53]:
for tag in sorted_thread_tags:
    if 'linux' in tag[3]:
        print(tag)

(244, 44, 244, 'linux')
(3, 2, 3, 'linuxgnueabihf')
(2, 2, 2, 'flash_linux_m4_ddr')
(2, 2, 2, 'flash_linux_m4')
(2, 1, 2, 'linux.tar.bz2')
(1, 1, 1, 'selinux')
(1, 1, 1, 'pokylinux')
(1, 1, 1, 'mounting_a_windows_share_in_linux')
(1, 1, 1, 'linuxfoundation.org')
(1, 1, 1, 'linux_version_extension')
(1, 1, 1, 'linux64')
(1, 1, 1, 'l4.14.98_2.3.0_linux_docs')
(1, 1, 1, 'l4.14.98_2.2.0_linux_docs')
(1, 1, 1, 'i.mx_linux_users_guide')
(1, 1, 1, 'elinux.org')
(1, 1, 1, 'buildinganddeployingtheproductionlinuxbsp')
(1, 1, 1, 'buildinganddeployinglinuxembedded')
(1, 1, 1, '2flinux_customer_readme')


In [111]:
index='tasks'
keyword = 'i.mx8qm'
fields=['project','task','task_description']

es.search(index=index, body={
    'query':{
        'multi_match':{
            'query': keyword,
            'fields': fields
        }
    }
})

{'took': 3,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 7, 'relation': 'eq'},
  'max_score': 3.9853926,
  'hits': [{'_index': 'tasks',
    '_type': '_doc',
    '_id': 'nZCC8m8BTANBIwoUsx-l',
    '_score': 3.9853926,
    '_source': {'project_id': 'arv',
     'internal_project_id': 2100339,
     'project_time_budget': 50,
     'tasklist': 'i.mx8qm ddr debug',
     'internal_tasklist_id': 3847340,
     'task': 'i.mx8qm ddr debug',
     'task_id': 'arv-1',
     'internal_task_id': 16651107,
     'task_description': 'https://community.nxp.com/thread/512109',
     'worked_time': '43 min',
     'decimal_hours': 0.73,
     'time_in_seconds': 2619}},
   {'_index': 'tasks',
    '_type': '_doc',
    '_id': 'npCC8m8BTANBIwoUsx-l',
    '_score': 3.9853926,
    '_source': {'project_id': 'arv',
     'internal_project_id': 2100339,
     'project_time_budget': 50,
     'tasklist': 'i.mx8qm ddr debug',
     'internal_tasklist_id

In [15]:
next(generate_combined_tags(index='threads', 
                               ids='hBKP728BJn_G0KX-yeNc', fields=['text']))

('2000', {'doc_freq': 2, 'ttf': 2, 'term_freq': 1})

In [125]:
'0x'.startswith('0x')

True